In [1]:
from src.datasets.Dataset_PESO import Dataset_PESO
from src.utils.Study import Study
from src.utils.ProjectConfiguration import ProjectConfiguration
from src.utils.BaselineConfigs import EXP_OctreeNCA
from src.datasets.Dataset_BCSS_Seg import Dataset_BCSS_Seg
from src.datasets.Dataset_AGGC import Dataset_AGGC
import octree_vis, torch, os, json, openslide, math
import einops
from src.models.Model_OctreeNCAV2 import OctreeNCAV2
import numpy as np
import torch.nn.functional as F
from src.utils.ProjectConfiguration import ProjectConfiguration as pc
from src.models.Model_OctreeNCA_2d_patching2 import OctreeNCA2DPatch2
import matplotlib.pyplot as plt

torch.set_grad_enabled(False)

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
model_path = "/local/scratch/clmn1/octree_study_new/Experiments/pesoLargeGroupNorm_OctreeNCA2DSegmentation"
model_path = "/local/scratch/clmn1/octree_study_new/Experiments/peso_med_OctreeNCA2DSegmentation/"


with open(os.path.join(model_path, "config.json")) as f:
    config = json.load(f) 

exp = EXP_OctreeNCA().createExperiment(config, detail_config={}, 
                                                      dataset_class=Dataset_PESO, dataset_args={
                                                            'patches_path': os.path.join(pc.FILER_BASE_PATH, config['experiment.dataset.patches_path']),
                                                            'patch_size': config['experiment.dataset.input_size'],
                                                            'path': os.path.join(pc.FILER_BASE_PATH, config['experiment.dataset.img_path']),
                                                            'img_level': config['experiment.dataset.img_level']
                                                      })

model: OctreeNCA2DPatch2 = exp.model
assert isinstance(model, OctreeNCA2DPatch2)
model = model.eval().cpu()
model.device = "cpu"
for backbone in model.backbone_ncas:
    backbone.device = "cpu"

loading patches
Loaded configuration is missing keys: '['experiment.run_hash']'. Check if you are loading the correct experiment.
Reload State 2000
loading patches
loading patches

-------- Experiment Setup --------
{
    "experiment.name": "peso_med",
    "experiment.description": "OctreeNCA2DSegmentation",
    "model.output_channels": 1,
    "model.channel_n": 16,
    "model.fire_rate": 0.5,
    "model.kernel_size": [
        3,
        7
    ],
    "model.hidden_size": 64,
    "model.batchnorm_track_running_stats": false,
    "model.train.patch_sizes": [
        null,
        null
    ],
    "model.train.loss_weighted_patching": false,
    "model.eval.patch_wise": false,
    "model.octree.res_and_steps": [
        [
            [
                320,
                320
            ],
            20
        ],
        [
            [
                80,
                80
            ],
            40
        ]
    ],
    "model.octree.separate_models": true,
    "model.vitca": fals

In [4]:
def remove_names(x: torch.Tensor):
    x.names = [None] * len(x.names)
    return x

def align_tensor_to(x: torch.Tensor, target: str):
    if isinstance(target, tuple):
        target_str = ' '.join(target)
    elif isinstance(target, str): 
        if max(map(len, target.split())) != 1:
            #targets are like "BCHW"
            target_str = ' '.join(target)
        else:
            #targets are like "B C H W"
            target_str = target
            target = target.replace(" ", "")


    pattern = f"{' '.join(x.names)} -> {target_str}"
    x = remove_names(x)
    x = einops.rearrange(x, pattern)
    x.names = tuple(target)
    return x

def downscale(x: torch.Tensor, out_size):
    x = align_tensor_to(x, "BCHW")
    remove_names(x)

    out = F.interpolate(x, size=out_size)
    out.names = ('B', 'C', 'H', 'W')
    x.names = ('B', 'C', 'H', 'W')
    return out

In [5]:
def compute_resolutions(x_shape, model):
    upscale_factors = []
    for i in range(len(model.octree_res)-1):
        t = []
        for c in range(2):
            t.append(model.octree_res[i][c]//model.octree_res[i+1][c])
        upscale_factors.append(t)

    new_octree_res = [tuple(x_shape)]
    for i in range(1, len(model.octree_res)):
        downsample_factor = np.array(model.octree_res[i-1]) / np.array(model.octree_res[i])
        new_octree_res.append([math.ceil(new_octree_res[i-1][0] / downsample_factor[0]), 
                                math.ceil(new_octree_res[i-1][1] / downsample_factor[1])])
    return new_octree_res

In [ ]:
subject = "14"
pos_x, pos_y = 14400, 24320
#pos_x, pos_y = 14400 - 1000, 24320
size = (16*161, 16*161)
#size = (320, 320)

slide = openslide.open_slide(f"/local/scratch/PESO/peso_training/pds_{subject}_HE.tif")
slide = slide.read_region((int(pos_x * slide.level_downsamples[1]),
                           int(pos_y * slide.level_downsamples[1])), 1, size)
#slide = slide.read_region((int(pos_x * slide.level_downsamples[1]),
#                           int(pos_y * slide.level_downsamples[1])), 1, (16*10, 16*10))
slide = np.array(slide)[:,:,0:3]
slide_cpu = slide

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
slide = slide / 255.0
slide = (slide - mean) / std

slide = slide[None]
slide = torch.from_numpy(slide).float()
slide.names = ('B', 'H', 'W', 'C')

slide = align_tensor_to(slide, "BHWC")
computed_resolutions = compute_resolutions(slide.shape[1:3], model)
print(computed_resolutions)

seed = torch.zeros(1, *computed_resolutions[-1], model.channel_n,
                                dtype=torch.float, device=slide.device,
                                names=('B', 'H', 'W', 'C'))
temp = downscale(slide, computed_resolutions[-1])
temp = align_tensor_to(temp, "BHWC")
remove_names(temp)
remove_names(seed)
seed[:,:,:,:model.input_channels] = temp
#temp.names = ('B', 'H', 'W', 'C')
#seed.names = ('B', 'H', 'W', 'C')

((47484, 105289), (23742, 52644), (11871, 26322), (5935, 13161), (2967, 6580), (1483, 3290), (741, 1645), (370, 822))


/tmp/ipykernel_1527732/427445862.py:23: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1928.)
  slide.names = ('B', 'H', 'W', 'C')


NameError: name 'align_tensor_to' is not defined

In [7]:
state = model.backbone_ncas[4](seed, steps=model.inference_steps[4], fire_rate=model.fire_rate)

state = einops.rearrange(state, "B H W C -> B C H W")
state = torch.nn.Upsample(size=computed_resolutions[3], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(slide, "B H W C -> B C H W"), size=computed_resolutions[3])
state[0,:model.input_channels,:,:] = temp[0]
state = einops.rearrange(state, "B C H W -> B H W C")
state = model.backbone_ncas[3](state, steps=model.inference_steps[3], fire_rate=model.fire_rate)


state = einops.rearrange(state, "B H W C -> B C H W")
state = torch.nn.Upsample(size=computed_resolutions[2], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(slide, "B H W C -> B C H W"), size=computed_resolutions[2])
state[0,:model.input_channels,:,:] = temp[0]
state = einops.rearrange(state, "B C H W -> B H W C")
state = model.backbone_ncas[2](state, steps=model.inference_steps[3], fire_rate=model.fire_rate)


state = einops.rearrange(state, "B H W C -> B C H W")
state = torch.nn.Upsample(size=computed_resolutions[2], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(slide, "B H W C -> B C H W"), size=computed_resolutions[2])
state[0,:model.input_channels,:,:] = temp[0]
state = einops.rearrange(state, "B C H W -> B H W C")
state = model.backbone_ncas[2](state, steps=model.inference_steps[2], fire_rate=model.fire_rate)


state = einops.rearrange(state, "B H W C -> B C H W")
state = torch.nn.Upsample(size=computed_resolutions[1], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(slide, "B H W C -> B C H W"), size=computed_resolutions[1])
state[0,:model.input_channels,:,:] = temp[0]
state = einops.rearrange(state, "B C H W -> B H W C")
state = model.backbone_ncas[1](state, steps=model.inference_steps[1], fire_rate=model.fire_rate)


state = einops.rearrange(state, "B H W C -> B C H W")
state = torch.nn.Upsample(size=computed_resolutions[0], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(slide, "B H W C -> B C H W"), size=computed_resolutions[0])
state[0,:model.input_channels,:,:] = temp[0]
state = einops.rearrange(state, "B C H W -> B H W C")
state = model.backbone_ncas[0](state, steps=model.inference_steps[0], fire_rate=model.fire_rate)

IndexError: index 4 is out of range

In [10]:
state = seed
state = model.backbone_ncas[1](state, steps=model.inference_steps[1], fire_rate=model.fire_rate)


state = einops.rearrange(state, "B H W C -> B C H W")
state = torch.nn.Upsample(size=computed_resolutions[0], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(slide, "B H W C -> B C H W"), size=computed_resolutions[0])
state[0,:model.input_channels,:,:] = temp[0]
state = einops.rearrange(state, "B C H W -> B H W C")
state = model.backbone_ncas[0](state, steps=model.inference_steps[0], fire_rate=model.fire_rate)